# Rebalancing evaluation

In this notebook have been considered three different techniques to approach the imbalanced dataset:
- Oversampling: it consists in duplicating samples in the minority classes by picking them with replacement randomly
- Undersampling : it undersample the majority classes by randomly picking samples without replacement
- SMOTE: it generates synthetic examples from the existing samples of the minority classes

The executions take into account the six algorithms (LinearSVC, MultinomialNB, LogisticRegression, RandomForestClassifier, KNeighborsClassifier, BernulliNB) considered with the best tuned parameters performed in the notebook 'Classifier evaluation' and a step of rebalancing was put into the pipeline. 
The comparison is done comparing not only the results of the rebalancing methods but also with those occured without rebalancing; therefore these results will be reported at the beginning of each model paragraph.

At the end of this notebook is shown a summary table with all the executions' outcomes.

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import nltk
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer


#Load pre-processed dataset
data = pd.read_csv("movies/data.csv", encoding="ISO-8859-1")
data = data.dropna(subset = ["overview"])
data = data.reset_index(drop=True)

encoder = LabelEncoder()
data['genre_ids'] = encoder.fit_transform(data.genre_ids)

X = data['overview']
y = data['genre_ids']

class LemmaTokenizer:
     def __init__(self):
            self.stm = SnowballStemmer("english")
     def __call__(self, doc):
        return [self.stm.stem(t) for t in word_tokenize(doc)]

oversampling=[]
sm=[]
undersampling=[]
no_balance=[]

In [4]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from imblearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import chi2, f_classif
from sklearn.model_selection import GridSearchCV

n_folds = 10
skf = StratifiedKFold(n_folds, shuffle=True)

scoring = ['accuracy', 'precision_micro', 'recall_micro', 'f1_micro']

# Vectorization parameters
# N-gram sizes for tokenizing text.
NGRAM_RANGE = [(1,2)]
# Minimum document frequency below which a token will be discarded.
MIN_DOCUMENT_FREQUENCY =  [2]

## LinearSVC
Best estimator selected is that with 20000 features and the chi2 score function with a best score of 0.5967010468382352.

In [3]:
from sklearn.svm import LinearSVC
   
grid = {
    'tfidfvectorizer__min_df': MIN_DOCUMENT_FREQUENCY,
    'tfidfvectorizer__ngram_range': NGRAM_RANGE,
    'tfidfvectorizer__tokenizer': [LemmaTokenizer()],
    'selectkbest__k': [20000],
    'selectkbest__score_func': [chi2]  
    }
no_balance.append(0.5967010468382352)

In [4]:
ros_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomOverSampler(random_state=1),
                        SelectKBest(),
                        LinearSVC())

grid_search = GridSearchCV(ros_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
svc_ros_model = grid_search.fit(X=X, y=y)

svc_ros_model.best_score_

0.5709483157752279

In [5]:
smote_pipe = make_pipeline(TfidfVectorizer(), 
                        SMOTE(sampling_strategy='minority', k_neighbors=5, random_state=1),
                        SelectKBest(),
                        LinearSVC())

grid_search = GridSearchCV(smote_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
svc_smote_model = grid_search.fit(X=X, y=y)


svc_smote_model.best_score_ 

0.5892214007070729

In [6]:
rus_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomUnderSampler(random_state=1),
                        SelectKBest(),
                        LinearSVC())

grid_search = GridSearchCV(rus_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
svc_rus_model = grid_search.fit(X=X, y=y)

svc_rus_model.best_score_ 

0.51878472741605

In [7]:
oversampling.append(svc_ros_model.best_score_)
sm.append(svc_smote_model.best_score_)
undersampling.append(svc_rus_model.best_score_)

## MultinomialNB
Best estimator selected is that with 5000 features and the f_classif (default) score function with a best score of 0.4810446123089561.

In [4]:
from sklearn.naive_bayes import MultinomialNB

grid = {
    'tfidfvectorizer__min_df': MIN_DOCUMENT_FREQUENCY,
    'tfidfvectorizer__ngram_range': NGRAM_RANGE,
    'tfidfvectorizer__tokenizer': [LemmaTokenizer()],
    'selectkbest__k': [5000],
    'selectkbest__score_func': [f_classif]  
    }

no_balance.append(0.4810446123089561)

In [9]:
ros_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomOverSampler(random_state=1),
                        SelectKBest(),
                        MultinomialNB())

grid_search = GridSearchCV(ros_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
mnb_ros_model = grid_search.fit(X=X, y=y)


mnb_ros_model.best_score_

0.5581330890156776

In [10]:
smote_pipe = make_pipeline(TfidfVectorizer(), 
                        SMOTE(sampling_strategy='minority', k_neighbors=5, random_state=1),
                        SelectKBest(),
                        MultinomialNB())

grid_search = GridSearchCV(smote_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
mnb_smote_model = grid_search.fit(X=X, y=y)

mnb_smote_model.best_score_

0.4966620604834593

In [5]:
rus_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomUnderSampler(random_state=1),
                        #VarianceThreshold(),
                        SelectKBest(),
                        MultinomialNB())

grid_search = GridSearchCV(rus_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
mnb_rus_model = grid_search.fit(X=X, y=y)

mnb_rus_model.best_score_

e:\program\miniconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [0 0 0 ... 0 0 0] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
e:\program\miniconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


0.5302845861439363

In [14]:
oversampling.append(mnb_ros_model.best_score_)
sm.append(mnb_smote_model.best_score_)
undersampling.append(mnb_rus_model.best_score_)

## LogisticRegression 
Best estimator selected is that with all the features, the f_classif (default) score function and the default solver that is 'lbfgs' with a best score of 0.5945327988736221.

In [7]:
from sklearn.linear_model import LogisticRegression

grid = {
    'tfidfvectorizer__min_df': MIN_DOCUMENT_FREQUENCY,
    'tfidfvectorizer__ngram_range': NGRAM_RANGE,
    'tfidfvectorizer__tokenizer': [LemmaTokenizer()], 
    }

no_balance.append(0.5945327988736221) 

In [29]:
ros_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomOverSampler(random_state=1),
                        LogisticRegression(max_iter=1000, multi_class='ovr'))

grid_search = GridSearchCV(ros_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
lr_ros_model = grid_search.fit(X=X, y=y)

lr_ros_model.best_score_

0.3603452381800353

In [31]:
smote_pipe = make_pipeline(TfidfVectorizer(), 
                        SMOTE(sampling_strategy='minority', k_neighbors=5, random_state=1),
                        LogisticRegression(max_iter=1000, multi_class='ovr'))

grid_search = GridSearchCV(smote_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
lr_smote_model = grid_search.fit(X=X, y=y)

lr_smote_model.best_score_

0.3780819580077545

In [8]:
rus_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomUnderSampler(random_state=1),
                        LogisticRegression(max_iter=1000, multi_class='ovr'))

grid_search = GridSearchCV(rus_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
lr_rus_model = grid_search.fit(X=X, y=y)

lr_rus_model.best_score_ 

0.5429295681347095

In [34]:
oversampling.append(lr_ros_model.best_score_)
sm.append(lr_smote_model.best_score_)
undersampling.append(lr_ros_model.best_score_)

## RandomForest
Best estimator selected is that with 500 features, chi2 score function and a max tree depth of 8 with a best score of 0.38202870316948145.

In [42]:
from sklearn.ensemble import RandomForestClassifier

grid = {
    'tfidfvectorizer__min_df': MIN_DOCUMENT_FREQUENCY,
    'tfidfvectorizer__ngram_range': NGRAM_RANGE,
    'tfidfvectorizer__tokenizer': [LemmaTokenizer()],
    'selectkbest__k': [500],
    'selectkbest__score_func': [chi2],
    'randomforestclassifier__max_depth': [8],
    'randomforestclassifier__n_estimators': [100]
    }

no_balance.append(0.38202870316948145)

In [19]:
ros_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomOverSampler(random_state=1),
                        SelectKBest(),
                        RandomForestClassifier())

grid_search = GridSearchCV(ros_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
rf_ros_model = grid_search.fit(X=X, y=y)

rf_ros_model.best_score_

0.40115503605406805

In [20]:
smote_pipe = make_pipeline(TfidfVectorizer(), 
                        SMOTE(sampling_strategy='minority', k_neighbors=5, random_state=1),
                        SelectKBest(),
                        RandomForestClassifier())

grid_search = GridSearchCV(smote_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
rf_smote_model = grid_search.fit(X=X, y=y)

rf_smote_model.best_score_

0.3731849263841699

In [21]:
rus_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomUnderSampler(random_state=1),
                        SelectKBest(),
                        RandomForestClassifier())

grid_search = GridSearchCV(rus_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
rf_rus_model = grid_search.fit(X=X, y=y)

rf_rus_model.best_score_

0.3994731143734284

In [43]:
oversampling.append(rf_ros_model.best_score_)
sm.append(rf_smote_model.best_score_)
undersampling.append(rf_rus_model.best_score_)

## KNeighborsClassifier
Best estimator selected is that with all the features and the default f_classif score function with a best score of 0.447885376080613.

In [5]:
from sklearn.neighbors import KNeighborsClassifier

grid = {
    'tfidfvectorizer__min_df': MIN_DOCUMENT_FREQUENCY,
    'tfidfvectorizer__ngram_range': NGRAM_RANGE,
    'tfidfvectorizer__tokenizer': [LemmaTokenizer()]
}

no_balance.append(0.447885376080613)

In [6]:
ros_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomOverSampler(random_state=1),
                        KNeighborsClassifier())

grid_search = GridSearchCV(ros_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
knc_ros_model = grid_search.fit(X=X, y=y)

knc_ros_model.best_score_

0.34884509453360024

In [7]:
smote_pipe = make_pipeline(TfidfVectorizer(), 
                        SMOTE(sampling_strategy='minority', k_neighbors=5, random_state=1),
                        KNeighborsClassifier())

grid_search = GridSearchCV(ros_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
knc_smote_model = grid_search.fit(X=X, y=y)

knc_smote_model.best_score_

0.347164003865395

In [8]:
rus_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomUnderSampler(random_state=1),
                        KNeighborsClassifier())

grid_search = GridSearchCV(ros_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
knc_rus_model = grid_search.fit(X=X, y=y)

knc_rus_model.best_score_

0.3454345185944968

In [9]:
oversampling.append(knc_ros_model.best_score_)
sm.append(knc_smote_model.best_score_)
undersampling.append(knc_rus_model.best_score_)

## BernoulliNB Classifier
Best estimator selected is that with 10000 features and the f_classif(default) score function with a best score of 0.5815959237652936.

In [58]:
from sklearn.naive_bayes import BernoulliNB

grid = {
    'tfidfvectorizer__min_df': MIN_DOCUMENT_FREQUENCY,
    'tfidfvectorizer__ngram_range': NGRAM_RANGE,
    'tfidfvectorizer__tokenizer': [LemmaTokenizer()],
    'selectkbest__k': [10000]
    }

no_balance.append(0.5815959237652936)

In [59]:
ros_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomOverSampler(random_state=1),
                        SelectKBest(),
                        BernoulliNB())

grid_search = GridSearchCV(ros_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
bnc_ros_model = grid_search.fit(X=X, y=y)

bnc_ros_model.best_score_

0.5771124514154516

In [60]:
smote_pipe = make_pipeline(TfidfVectorizer(), 
                        SMOTE(sampling_strategy='minority', k_neighbors=5, random_state=1),
                        SelectKBest(),
                        BernoulliNB())

grid_search = GridSearchCV(ros_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
bnc_smote_model = grid_search.fit(X=X, y=y)

bnc_smote_model.best_score_

0.5782579011475095

In [61]:
rus_pipe = make_pipeline(TfidfVectorizer(), 
                        RandomUnderSampler(random_state=1),
                        SelectKBest(),
                        BernoulliNB())

grid_search = GridSearchCV(ros_pipe, param_grid=grid, scoring=scoring, refit="f1_micro", n_jobs=-1, cv=skf)
bnc_rus_model = grid_search.fit(X=X, y=y)

bnc_rus_model.best_score_

0.5789882482970181

In [62]:
oversampling.append(bnc_ros_model.best_score_)
sm.append(bnc_smote_model.best_score_)
undersampling.append(bnc_rus_model.best_score_)

In [63]:
methods=['Oversampling','Smote', 'Undersampling', 'No balance']
models=['LinearSVC',  'MultinomialNB', 'LogisticRegression', 'RandomForest', 'KNeighbors', 'BernoulliNB']
d = {'Oversampling': oversampling, 'SMOTE': sm, 'Undersampling': undersampling, 'No balance': no_balance}
df = pd.DataFrame(d, columns=methods, index=models)

df

,Oversampling,Smote,Undersampling,No balance
LinearSVC,0.573336,0.591390,0.523560,0.596701
MultinomialNB,0.559156,0.496126,0.530261,0.481045
LogisticRegression,0.360345,0.378082,0.360345,0.594533
RandomForest,0.401155,0.373185,0.399473,0.382029
KNeibghorsC,NaN,NaN,NaN,0.447885
BernoulliNC,0.577112,0.578258,0.578988,0.581596
